In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import tensorflow as tf
import json
import os
import import_ipynb
from Model import GoogLeNet

importing Jupyter notebook from Model.ipynb


In [2]:
data_root = os.path.abspath(os.path.join(os.getcwd(),"../../datasets"))
image_path = data_root+"/flower_data/"
train_dir = image_path+'train'
validation_dir = image_path+'val'

In [3]:
if not os.path.exists('save_weights'):
    os.mkdir('save_weights')
im_height=224
im_width =224
batch_size=32
epochs = 30

In [4]:
def pre_function(img):
    img = img/255.
    img = (img-0.5)*2.0
    return img

In [5]:
train_image_generator = ImageDataGenerator(preprocessing_function=pre_function,
                                          horizontal_flip=True)
validation_image_generator = ImageDataGenerator(preprocessing_function=pre_function)


train_data_gen = train_image_generator.flow_from_directory(directory=train_dir,
                                                          batch_size=batch_size,
                                                          shuffle=True,
                                                          target_size=(im_height,im_width),
                                                          class_mode='categorical')
total_train = train_data_gen.n

Found 3306 images belonging to 5 classes.


In [6]:
class_indices = train_data_gen.class_indices
class_indices

{'daisy': 0, 'dandelion': 1, 'roses': 2, 'sunflowers': 3, 'tulips': 4}

In [7]:
inverse_dict = dict((val,key) for key,val in class_indices.items())

In [8]:
json_str = json.dumps(inverse_dict,indent=4)
with open('class_indices.json','w') as json_file:
    json_file.write(json_str)

In [9]:
val_data_gen = validation_image_generator.flow_from_directory(directory=validation_dir,
                                                             batch_size=batch_size,
                                                             shuffle=False,
                                                             target_size=(im_height,im_width),
                                                             class_mode='categorical')
total_val = val_data_gen.n

Found 364 images belonging to 5 classes.


In [10]:
model = GoogLeNet(im_height=im_height,im_width=im_width,class_num=5,aux_logits=True)
# model.build((batch_size,224,224,3)) # subclass model
model.summary()

AttributeError: 'function' object has no attribute 'shape'

In [21]:
loss_object = tf.keras.losses.CategoricalCrossentropy(from_logits=False)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0003)

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.CategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.CategoricalAccuracy(name='test_accuracy')

In [22]:
@tf.function
def train_step(images,lables):
    with tf.GradientTape() as tape:
        aux1,aux2,output = model(images,training=True)
        loss1 = loss_object(lables,aux1)
        loss2 = loss_object(lables,aux2)
        loss3 = loss_object(lables,output)
        loss = loss1*0.3 + loss2*0.3 + loss3
    gradients = tape.gradient(loss,model.trainable_variables)
    optimizer.apply_gradients(zip(gradients,model.trainable_variables))
    
    train_loss(loss)
    train_accuracy(lables,output)

In [17]:
@tf.function
def test_step(images,labels):
    _,_,output = model(images,training=False)
    t_loss = loss_object(labels,output)
    
    test_loss(t_loss)
    test_accuracy(labels,output)

In [18]:
best_test_loss = float('inf')
for epoch in range(1,epochs+1):
    train_loss.reset_states()
    test_loss.reset_states()
    train_accuracy.reset_states()
    test_accuracy.reset_states()
    
    for step in range(total_train//batch_size):
        images,labels = next(train_data_gen)
        train_step(images,labels)
        
    for step in range(total_val//batch_size):
        test_images,test_labels = next(val_data_gen)
        test_step(test_images,test_labels)
    
    template ='Epoch {}, Loss:{}, Accuracy:{}, Test Loss:{}, Test Accuracy:{}'
    print(template.format(epoch,
                         train_loss.result(),
                         train_accuracy.result()*100,
                         test_loss.result(),
                         test_accuracy.result()*100))
    if test_loss.result() < best_test_loss:
        best_test_loss=test_loss.result()
        model.save_weights('./save_weights/myGoogleNetCPU.h5')

Epoch 1, Loss:2.195668935775757, Accuracy:35.21685791015625, Test Loss:1.1382713317871094, Test Accuracy:46.306819915771484
Epoch 2, Loss:1.7295641899108887, Accuracy:50.97087478637695, Test Loss:0.9794753789901733, Test Accuracy:62.95180892944336
Epoch 3, Loss:1.4911433458328247, Accuracy:60.904090881347656, Test Loss:0.8690599799156189, Test Accuracy:64.45783233642578
Epoch 4, Loss:1.3782355785369873, Accuracy:64.26390075683594, Test Loss:0.8264938592910767, Test Accuracy:67.77108001708984
Epoch 5, Loss:1.2864623069763184, Accuracy:67.56261444091797, Test Loss:0.919053852558136, Test Accuracy:65.66265106201172
Epoch 6, Loss:1.2512352466583252, Accuracy:68.87599182128906, Test Loss:0.7279787659645081, Test Accuracy:73.79518127441406
Epoch 7, Loss:1.1951675415039062, Accuracy:72.08307647705078, Test Loss:0.7642440795898438, Test Accuracy:70.78313446044922
Epoch 8, Loss:1.118859052658081, Accuracy:72.72449493408203, Test Loss:0.7976974248886108, Test Accuracy:70.78313446044922
Epoch 9, 